In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale

In [2]:
train = pd.read_csv('train.csv')

In [3]:
indep = train.columns[:-1]

In [4]:
discrete = []
continuous = []
for v in indep:
    if train[v].dtype == 'object':
        discrete.append(v)
    else:
        continuous.append(v)

In [5]:
train[continuous] = scale(train[continuous])

In [6]:
dummy = pd.get_dummies(train[discrete])
X = pd.concat([train[continuous], dummy], axis=1)

In [7]:
dep = train.columns[-1]
y = train[dep]

In [8]:
X.shape

(24999, 46)

In [9]:
X.head()

,age,education,capital_gain,capital_loss,hours_per_week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,sex_Female,sex_Male
0,0.039911,1.131185,0.143593,-0.21803,-0.075261,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,0.874716,1.131185,-0.146659,-0.21803,-2.336225,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,-0.035980,-0.441896,-0.146659,-0.21803,-0.075261,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,1.102390,-1.228437,-0.146659,-0.21803,-0.075261,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,-0.794893,1.131185,-0.146659,-0.21803,-0.075261,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


## Neural Network를 위한 replace value 0,1

In [10]:
y = y.replace(['under50k','over50k'],[0,1])

In [11]:
X = X.as_matrix()

In [12]:
y = y.as_matrix()

In [13]:
y = np.asmatrix(y).T

In [14]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

In [17]:
y_train[1:10]

matrix([[0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]], dtype=int64)

# Tensorflow 불러오기

In [18]:
import tensorflow as tf
import tensorlayer as tl

### 기본 그래프 만들기

In [106]:
tf.reset_default_graph()
tl.layers.set_name_reuse(enable=True)

### 신경망 만들기

In [107]:
x = tf.placeholder(tf.float32, [None, 46]) # 입력 데이터

In [108]:
network = tl.layers.InputLayer(x,name="input") # 입력층

  tensorlayer:Instantiate InputLayer  input: (?, 46)


In [109]:
network = tl.layers.DropoutLayer(network, keep=0.8, name='drop1')

  tensorlayer:Instantiate DropoutLayer drop1: keep: 0.800000


In [110]:
network = tl.layers.DenseLayer(network,n_units=15,act=tf.sigmoid, name="hidden")

  tensorlayer:Instantiate DenseLayer  hidden: 15, sigmoid


In [111]:
network = tl.layers.DenseLayer(network,n_units=10,act=tf.sigmoid, name="hidden2")

  tensorlayer:Instantiate DenseLayer  hidden2: 10, sigmoid


In [112]:
network = tl.layers.DenseLayer(network, n_units=1, act=tf.sigmoid, name="output") # 출력층

  tensorlayer:Instantiate DenseLayer  output: 1, sigmoid


### 데이터

### 비용

In [113]:
predict = network.outputs # 예측값
y = tf.placeholder(tf.float32, [None, 1]) # 실제 값
cost = tl.cost.binary_cross_entropy(network.outputs, y)

In [114]:
gd = tf.train.RMSPropOptimizer(learning_rate=0.1, momentum=0.1)
train_step = gd.minimize(cost)  # cost를 최소화한다

## 데이터 설정

In [115]:
data = {x:X_train,y:y_train}

### 세션과 변수 초기화

In [116]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

### 그래프를 세션에서 실행

In [117]:
predict.eval(data)  # 초기 예측값

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_1' with dtype float
	 [[Node: Placeholder_1 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Placeholder_1', defined at:
  File "C:\Anaconda3\envs\work\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Anaconda3\envs\work\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Anaconda3\envs\work\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Anaconda3\envs\work\lib\site-packages\traitlets\config\application.py", line 596, in launch_instance
    app.start()
  File "C:\Anaconda3\envs\work\lib\site-packages\ipykernel\kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "C:\Anaconda3\envs\work\lib\site-packages\zmq\eventloop\ioloop.py", line 162, in start
    super(ZMQIOLoop, self).start()
  File "C:\Anaconda3\envs\wor

In [102]:
cost.eval(data)

0.68765736

In [103]:
train_step.run(data)

In [104]:
cost.eval(data)

0.66501421

In [105]:
tl.utils.fit(sess, network, train_step, cost, X_train, y_train, x, y, 
            batch_size=50,  # 한 번에 30개의 데이터 학습
            n_epoch=1000,
            print_freq=100
            )

Start training the network ...
Epoch 1 of 1000 took 0.202252s, loss 0.404010
Epoch 100 of 1000 took 0.190738s, loss 0.339153
Epoch 200 of 1000 took 0.190738s, loss 0.348694
Epoch 300 of 1000 took 0.202253s, loss 0.350984
Epoch 400 of 1000 took 0.207759s, loss 0.351435
Epoch 500 of 1000 took 0.195745s, loss 0.351992
Epoch 600 of 1000 took 0.204756s, loss 0.347985
Epoch 700 of 1000 took 0.193241s, loss 0.348584
Epoch 800 of 1000 took 0.187735s, loss 0.357707
Epoch 900 of 1000 took 0.188235s, loss 0.356928
Epoch 1000 of 1000 took 0.209261s, loss 0.353617
Total training time: 199.034533s


In [56]:
y_predict = predict.eval({x: X_test})

In [57]:
y_class = y_predict > 0.5  # 0.5와 비교

In [58]:
y_class = y_class.astype(int)

## 평가

In [59]:
from sklearn import metrics

In [60]:
def getResult(y_test,y_pred):
    print(metrics.confusion_matrix(y_test, y_pred))
    print('accurracy:', metrics.accuracy_score(y_test, y_pred))
    print('precision:', metrics.precision_score(y_test, y_pred, pos_label=1))
    print('recall:', metrics.recall_score(y_test, y_pred, pos_label=1))
    print('f1:', metrics.f1_score(y_test, y_pred, pos_label=1))
    

     learn : 0.1 
     momentum : 0.1
     layer 15 10 1 
     sigmoid
     0.65

In [61]:
getResult(y_test,y_class)

[[1819   52]
 [ 385  244]]
accurracy: 0.8252
precision: 0.824324324324
recall: 0.387917329094
f1: 0.527567567568
